# 구조적 API 기본 연산
* DataFrame
  * Row 타입의 레코드와 컬럼으로 구성
* 스키마 
  * 각 컬럼명과 데이터 타입을 정의
* 파티셔닝
  * DataFrame이나 DataSet이 클러스터에서 물리적으로 배치되는 형태 정의
  * 분할 기준은 매번 변하는 값을 기반으로 설정
* 파티셔닝 스키마
  * 파티션을 배치하는 방법 정의
  
## 스키마
* DataFrame의 컬럼명과 테이터 타입을 정의
* 스키마를 얻거나 직접 정의할 수 있음
  * 데이터를 읽기 전 스키마를 정의해야 하는지 여부는 상황에 따라 다름
  * 스키마-온-리드는 대부분 잘 동작하나 정밀도 문제 (e.g. Long 타입을 Integer로 인식)로 ETL 작업에서는 직접 스키마 정의해야 함
* 여러 개의 `StructField` 타입 필드로 구성된 `StructType` 객체
* `StructField`는 이름, 데이터 타입, 컬럽 값이 없거나 null일 수 있는지 지정하는 boolean 값, 메타 데이터 지정 가능
* 스파크는 자체 데이터 타입 정보를 사용 
  * 프로그래밍 언어의 데이터 타입을 스파크 데이터 타입으로 설정할 수 없음
  
## 컬럼과 표현식
* 표현식으로 DataFrame의 컬럼을 선택, 조작, 제거할 수 있음
* 컬럼은 논리적인 구조
  * 실제 값을 얻기 위해서는 DataFrame의 로우가 필요

### 컬럼
* 컬럼을 생성하고 참조하기 위해서 `col`함수나 `column`함수 사용
  * 스칼라의 경우 `$"column"` 또는 `'column` 형태로 컬럼을 참조할 수 있으나, 코드가 짧아진다고 성능이 좋아지는 것은 아님
* 카탈로그에 저장된 정보와 비교하기 전까지 컬럼이 DataFrame에 있는지 미확인 상태임
* `col` 메서드를 사용해 명시적으로 칼럼을 정의하면 스파크 분석기 실행 단계에서 컬럼 확인 절차 생략

### 표현식
* 레코드의 여러 값에 대한 트랜스포메이션 집합
* `expr`함수로 사용할 수 있음
* 컬럼은 표현식임
* 컬럼과 컬럼의 트랜스포메이션은 파싱된 표현식과 동일한 논리적 실행 계획으로 컴파일됨
* SQL 표현식과 DataFrame 코드는 실행 시점에 동일한 논리 트리로 컴파일되어 동일한 성능을 발휘
* 아래 코드는 그림의 지향성 비순환 그래프 (Dag)로 표현 가능하며, 동일
![]()

In [ ]:
from pyspark.sql.functions import col, expr

(((col("column1") + 5 ) * 200) - 6) < col("column2")
expr("((((column1 + 5)) * 200) -6) < column2")

* `printSchema` 메서드로 DataFrame 전체 컬럼 정보 확인 가능
* 프로그래밍 방식으로 컬럼에 접근할 때에는 DataFrame의 `columns` 속성 사용

## 레코드와 로우
* DataFrame의 각 로우는 하나의 레코드임
* `Row` 객체로 표현
* `Row` 객체 내부에 바이트 배열을 가짐
  * 컬럼 표현식으로만 다룰 수 있음
  * 사용자에게 절대 노출되지 않음
* 드라이버에 개별 로우를 반환하는 명령은 하나 이상의 Row 타입을 반환

### 로우 생성하기
* 스키마 정보를 가지고 있지 않기 때문에, DataFrame의 스키마와 같은 순서로 값을 명시해야 함
* 로우 데이터에 접근하기 위해서는 원하는 위치를 지정하기만 하면 됨
  * 스칼라나 자바에서는 명시적으로 데이터 타입 지정해야 함
  * 파이썬이나 R에서는 올바른 데이터 타입으로 자동 변환됨

## DataFrame의 트랜스포메이션
### DataFrame 생성하기
* 원시 데이터 소스에서 DataFrame 생성할 수 있음
* Row 객체 가진 Seq 타입을 직접 변환해 DataFrame 생성할 수 있음

### select와 selectExpr
* `select` 함수와 `selectExpr` 함수를 사용할 경우 SQL 실행하는 것처럼 컬럼을 다룰 수 있음
* `Column` 객체와 문자열을 함께 섞여 쓸 경우 컴파일러 오류 발생
* `expr` 함수는 유연한 참조 방법으로, 단순 컬럼 참조나 문자열을 이용해 컬럼을 참조할 수 있음
* `select` 메서드에 `expr` 함수를 사용하는 패턴을 자주 사용해 `selectExpr` 메서드 제공
* `selectExpr` 사용 시 모든 유효한 비집계형 SQL 구문을 지정할 수 있음
  * 단 컬럼을 식별할 수 있어야 함

### 스파크 데이터 타입으로 변환하기
* 명시적인 값을 스파크에 전달해야 할 때에는 리터럴 (`lit` 함수) 사용

### 컬럼 추가하기
* 신규 컬럼을 추가하는 공식적인 방법은 `withColumn` 메서드를 사용하는 것임
* `withColumn`의 두 인수 중 하나는 컬럼명이고, 다른 하나는 값을 생성할 표현식임
* `withColumn` 사용 시 컬럼명을 변경할 수도 있음

### 컬럼명 변경하기
* `withColumnRenamed`메서드로 컬럼명 변경 가능
* `withColumnRenamed` 메서드 첫 번째 인수로 전달된 컬럼명을 두 번째 인수의 문자열로 변경

### 예약 문자와 키워드
* 공백이나 하이픈(-)과 같은 예약 문자는 컬럼명에 사용할 수 없음
* 예약 문자 사용 시 백틱 (\`) 문자를 이용해 이스케이핑해야 함
* 표현식에만 예약 문자 이스케이프 처리가 필요

### 대소문자 구분
* 기본적으로 대소문자를 가리지 않음
* `set spark.sql.caseSensitive true` 설정 시 대소문자 구분

### 컬럼 제거하기
* `drop`메서드 사용

### 컬럼의 데이터 타입 변경하기
* `cast` 메서드로 데이터 타입 변환

### 로우 필터링하기
* `where` 메서드나 `filter` 메서드 사용
* 두 메서드 모두 동일하게 동작
* 같은 표현식에 여러 필터를 적용해야 할 때, 차례대로 필터를 연결하고 판단은 스파크에 맡겨야 함

### 교유한 로우 얻기
* 고윳값을 얻으려면 `distinct` 메서드 사용

### 무작위 샘플 만들기
* 무작위 샘플 데이터를 얻으려면 `sample` 메서드 사용

### 임의 분할하기
* DataFrame을 임의 크기로 분할할 때 `randomSplit` 메서드 사용
* 임의성을 가지도록 설계되어 시드값을 반드시 설정해야 함
* 총합이 1이 되도록 각 DataFrame의 비율을 지정
  * 기본값을 0.25, 0.75
  
### 로우 합치기와 추가하기
* DataFrame은 불변성을 가져 레코드를 추가하려면 원본 DataFrame을 `union` 메서드를 통해 새로운 DataFrame과 통합해야 함
* 두 DataFrame은 동일한 스키마와 컬럼 수를 가져야 함

### 로우 정렬하기
* `sort`와 `orderBy` 메서드 사용해 정렬
* 두 메서드는 같은 방식으로 동작
* 모두 컬럼 표현식과 문자열을 사용할 수 있음
* 다수의 컬럼 지정 가능
* 기본 동작 방식은 오름차순
  * 정렬 기준 명확하게 지정하기 위해서는 `asc`, `desc` 함수 사용
* `asc_nulls_first`, `asc_nulls_last`, `desc_nulls_first`, `desc_nulls_last` 메서드 사용시 null 값 표시 기준 지정 가능
* 파티션별 정렬은 `sortWithinParitions` 메서드 사용 가능

### 로우 제한하기
* 로우 수를 제한하기 위해서 `limit` 메서드 사용 가능

### repartition과 coalesce
* `repartition` 메서드 호출 시 전체 데이터 셔플하며, 사용할 파티션 수가 현재 파티션 수보다 많거나 컬럼 기준으로 파티션 만드는 경우에만 사용해야 함
* `coalesce` 메서드는 전체 데이터를 셔플하지 않고 파티션을 병합하는 경우에 사용

### 드라이버로 로우 데이티ㅓ 수집하기
* 아래 연산을 드라이버로 데이터를 수집하는 연산임
  * 전체 DataFrame의 모든 데이터를 수집하는 `collect` 메서드
  * 상위 N개 로우를 반환하는 `take` 메서드
  * 여러 로우를 보기 좋게 출력하는 `show` 메서드
  * 전체 데이터셋에 대한 반복 처리를 하는 `toLocalIterator` 메서드
* 드라이버로 모든 데이터 컬렉션을 수집하는 작업은 큰 비용이 발생 (e.g. collect, toLocalIterator)